In [1]:
#randomly select some sm to spot checks





import pandas as pd
import sys
infile='/Users/irffanalahi/Research/Research_update/SoftRD/hashtable_stats/tieredApproach/OwnGrouptest/try1/someTcell/Tregs/Tregs_thresholdpos_allthresholdcombinations.txt'
currentct='Tregs'


howmanyrandomsampleperpart=5 #3

centerlinenumber=[20,50,100,500,1000]
correspondinggap=[2,2,10,20,100]

indf=pd.read_csv(infile,sep='\t')
indf.head()

,Filename,Average Value,Line Count
0,g1_Tregs_g2_cm4Tregs_-0.6_int_g1_Tregs_g2_em8T...,0.325,228
1,g1_Tregs_g2_cm4Tregs_-0.8_int_g1_Tregs_g2_em8T...,0.650,9
2,g1_Tregs_g2_cm4Tregs_-0.8_int_g1_Tregs_g2_em8T...,0.475,19
3,g1_Tregs_g2_cm4Tregs_-0.4_int_g1_Tregs_g2_em8T...,0.525,55
4,g1_Tregs_g2_cm4Tregs_-0.4_int_g1_Tregs_g2_em8T...,0.450,315


In [2]:
indf[indf['Line Count']>=1000].shape

(616, 3)

In [3]:

smlist=[]

for i in range(len(centerlinenumber)):

    tempdf=indf[(indf['Line Count']>=centerlinenumber[i]-correspondinggap[i]) & (indf['Line Count']<=centerlinenumber[i]+correspondinggap[i])]
    if tempdf.shape[0]<10:
        print(centerlinenumber[i])
        print('not enough')
        sys.exit(1)

    t25=tempdf['Average Value'].quantile(q=0.25)
    t75=tempdf['Average Value'].quantile(q=0.75)




    
    tempsmallerthan25perce=tempdf[tempdf['Average Value']<t25]
    tempgreaterthan75perce=tempdf[tempdf['Average Value']>t75]
    tempinthemiddle=tempdf[(tempdf['Average Value']>=t25) & (tempdf['Average Value']<=t75)]
    
    
    try:
        RANDOMtempsmallerthan25perce=tempsmallerthan25perce.sample(n=howmanyrandomsampleperpart,random_state=0)
    except:
        
        RANDOMtempsmallerthan25perce=tempsmallerthan25perce
        print("in except")
      
        print(centerlinenumber[i])
        print("RANDOMtempsmallerthan25perce")
      
    try:
        RANDOMtempgreaterthan75perce=tempgreaterthan75perce.sample(n=howmanyrandomsampleperpart,random_state=0)
    except:
        RANDOMtempgreaterthan75perce=tempgreaterthan75perce
        print("in except")
        print(centerlinenumber[i])
        print("RANDOMtempgreaterthan75perce")
    
    try:
        RANDOMtempinthemiddle=tempinthemiddle.sample(n=howmanyrandomsampleperpart,random_state=0)
    except:
        RANDOMtempinthemiddle=tempinthemiddle
        print("in except")
        print(centerlinenumber[i])
        print("RANDOMtempinthemiddle")


    tempcombined=pd.concat([RANDOMtempsmallerthan25perce,RANDOMtempgreaterthan75perce,RANDOMtempinthemiddle])

    
    if tempcombined['Filename'].is_unique==False:
        print(centerlinenumber[i])
        print('not unique')
        sys.exit(1)

    
    tempcombined['CenterLineNumber']=centerlinenumber[i]
    tempcombined['Cell Type']=currentct

   
    
    
    smlist.append(tempcombined)

    


In [4]:
smlistdf=pd.concat(smlist)
smlistdf.shape

(75, 5)

In [5]:
smlistdf=smlistdf.drop_duplicates(subset='Filename', keep="first")
smlistdf.shape

(75, 5)

In [6]:
smlistdf.reset_index(inplace=True,drop=True)
smlistdf.reset_index(inplace=True)

In [7]:
smlistdf['SM']='SM'+smlistdf['Cell Type']+"_"+smlistdf['index'].map(str)
smlistdf.shape

(75, 7)

In [8]:
smlistdf.head()

,index,Filename,Average Value,Line Count,CenterLineNumber,Cell Type,SM
0,0,g1_Tregs_g2_cm4Tregs_-0.8_int_g1_Tregs_g2_em8T...,0.425,19,20,Tregs,SMTregs_0
1,1,g1_Tregs_g2_cm4Tregs_-0.8_int_g1_Tregs_g2_em8T...,0.375,20,20,Tregs,SMTregs_1
2,2,g1_Tregs_g2_cm4Tregs_-0.8_int_g1_Tregs_g2_em8T...,0.425,18,20,Tregs,SMTregs_2
3,3,g1_Tregs_g2_cm4Tregs_-0.8_int_g1_Tregs_g2_em8T...,0.425,22,20,Tregs,SMTregs_3
4,4,g1_Tregs_g2_cm4Tregs_-0.2_int_g1_Tregs_g2_em8T...,0.450,19,20,Tregs,SMTregs_4


In [9]:
if smlistdf['SM'].is_unique==False:
   
    print('SM not unique')
    sys.exit(1)


In [10]:
smlistdf.to_csv(infile+"_fianlselection.txt",sep='\t',index=False)